In [ ]:
 # mount GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#in case previos cell does not work move one folder at a time....for me the cell above stopped working at some point!
%cd 'drive'
%cd 'My Drive'
%cd 'Colab Notebooks'
%cd 'Adversarial_LSBGs'
%cd 'AdversarialStudies-master'
%ls

/content/drive
/content/drive/My Drive
/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/Adversarial_LSBGs
/content/drive/My Drive/Colab Notebooks/Adversarial_LSBGs/AdversarialStudies-master
Adversarial_Tests.ipynb               LSBG1613148641.txt
check_coverage.sh                     LSBG1613148914.txt
cleanup.sh                            LSBG1613149120.txt
dset_viz_attacked_stargalaxy.py       LSBG1613149286.txt
dset_viz_stargalaxy.py                LSBG1613149647.txt
evts_0_010.pdf                        LSBG1613149961.txt
fgsm_0_000.hdf5                       LSBG1613150472.txt
fgsm_0_010.hdf5                       LSBG1613150586.txt
fgsm_0_050.hdf5                       LSBG_ckpt.tar
fgsm_attack_stargalaxy.py             LSBG_Outputs/
log_attack_LSBG1612999766.txt         models.py
log_attack_star_galaxy1612905554.txt  msg.py
log_star_galaxy1612810053.txt         notebooks/
log_star_galaxy1612903427.txt         ptlib/
log_star_galaxy1613081577.txt   

# Run this notebook second

This notebook will run the Fast Gradient Sign Method attack on a star-galaxy dataset, using a classifier trained in a different notebook. You will need to update some of the options (e.g., `data_dir`, possibly `ckpt_path` if you renamed it, etc.) below for your local situation. If you need a copy of the data, see the instructions at the root-level README.

The notebook will produce a set of HDF5 files containing images with the FGSM attack applied with names like `fgsm_0_XYZ.hdf5` for epsilon (attack "magnitude") 0.XYZ (float).

In [ ]:
import sys
import logging
import time

In [ ]:
sys.path.append("..")

In [ ]:
!pip install pathlib

In [ ]:
from ptlib.dataloaders import LSBGDataManager as DataManager
from ptlib.models import SimpleLSBGConvNet as Model
from ptlib.attackers import FGSMAttacker as Attacker
from ptlib.utils import get_logging_level
from ptlib.utils import log_function_args

In [ ]:
git_hash=!git describe --abbrev=12 --dirty --always
git_hash = git_hash[0]
print(git_hash)

fatal: not a git repository (or any parent up to mount point /content)


In [ ]:
ckpt_path="LSBG_ckpt.tar"
data_dir="../LSBG_Datasets/"
epsilons = [0.0, 0.01, 0.05]
log_freq=20
log_level="INFO"
short_test=False

Lots of useful information will go to the log file:

In [ ]:
logfilename = 'log_attack_LSBG' + str(int(time.time())) + '.txt'
print(logfilename)

log_attack_LSBG1613155894.txt


In [ ]:
logging.basicConfig(
    filename=logfilename, level=get_logging_level(log_level),
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
LOGGER = logging.getLogger(__name__)
LOGGER.info("Starting...")
log_function_args(vars())

Set up a data manager and a model:

In [ ]:
data_manager = DataManager(data_dir=data_dir)
data_manager.make_means()
model = Model()

Set up the attacker:

In [ ]:
attacker = Attacker(data_manager, model, ckpt_path, log_freq)
attacker.restore_model_and_optimizer()

Run the attack for each epsilon (FGSM scale factor):

In [ ]:
for epsilon in epsilons:
    attacker.attack_for_single_epsilon(
        epsilon, short_test=short_test)